In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

In [2]:
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

In [3]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)

In [4]:
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [5]:
print(wiki.name)
arxiv.name

wikipedia


'arxiv'

In [6]:
tools = [wiki,arxiv]

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
loader = WebBaseLoader("https://api.python.langchain.com/en/latest/langchain_api_reference.html")

In [9]:
docs = loader.load()

In [10]:
split_docs = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = split_docs.split_documents(docs)

In [11]:
embeddings = CohereEmbeddings(model="embed-english-v3.0")

In [12]:
embeddings.embed_documents([chunk.page_content for chunk in documents])

[[-0.0960083,
  0.0013999939,
  -0.059051514,
  -0.016616821,
  -0.019073486,
  0.026367188,
  -0.04196167,
  0.015151978,
  0.022598267,
  0.04699707,
  0.0049057007,
  0.024627686,
  0.015129089,
  -0.015914917,
  -0.009757996,
  0.010360718,
  0.042266846,
  -0.06500244,
  -0.03805542,
  -0.007675171,
  -0.052978516,
  0.024261475,
  -0.007507324,
  0.07977295,
  0.025161743,
  0.06286621,
  0.030685425,
  0.008171082,
  0.01259613,
  0.013130188,
  0.024917603,
  -0.047332764,
  -0.018325806,
  0.0826416,
  -0.04425049,
  -0.015792847,
  -0.010475159,
  0.04940796,
  0.008346558,
  0.016784668,
  0.0023918152,
  -0.010986328,
  0.03591919,
  -0.023284912,
  0.0005440712,
  -0.039001465,
  0.039489746,
  -0.011749268,
  0.00459671,
  -0.011726379,
  0.015014648,
  0.07659912,
  0.018615723,
  0.042419434,
  -0.006385803,
  0.014411926,
  -0.010101318,
  -0.02229309,
  0.016189575,
  0.041259766,
  0.008163452,
  0.02027893,
  0.005004883,
  -0.001332283,
  -0.0039634705,
  -0.028671

In [13]:
vectordb = FAISS.from_documents(documents,embeddings)

In [14]:
retriever = vectordb.as_retriever()

In [15]:
retreive_tool = create_retriever_tool(retriever,"langchain_search","Search any information about langchain")

In [16]:
retreive_tool.name

'langchain_search'

In [17]:
tools=[wiki,arxiv,retreive_tool]

In [18]:
from langchain_cohere import ChatCohere

In [27]:
llm = ChatCohere(model="command-r-plus")

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent

In [30]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert who answers the user's question with the most relevant datasource."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])


In [31]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True  # Needed for ChatPromptTemplate
)

In [32]:
agent = create_cohere_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

In [33]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [36]:
response = llm.invoke("Who is the Prime Minister of India? along with his full name")
print(response)

content='Shri Narendra Damodardas Modi is the current and 14th Prime Minister of India.' additional_kwargs={'id': 'cf0e33db-709f-4b0f-bb00-9fc70d39778b', 'finish_reason': 'COMPLETE', 'content': 'Shri Narendra Damodardas Modi is the current and 14th Prime Minister of India.', 'token_count': {'input_tokens': 79.0, 'output_tokens': 20.0}} response_metadata={'id': 'cf0e33db-709f-4b0f-bb00-9fc70d39778b', 'finish_reason': 'COMPLETE', 'content': 'Shri Narendra Damodardas Modi is the current and 14th Prime Minister of India.', 'token_count': {'input_tokens': 79.0, 'output_tokens': 20.0}} id='run--a3a397e7-3360-4db3-bc44-e8440a1b0142-0' usage_metadata={'input_tokens': 79, 'output_tokens': 20, 'total_tokens': 99}


In [37]:
response = llm.invoke("Who was the 1st")
print(response)

content='To answer your question, I need some additional context. Who or what are you referring to specifically?' additional_kwargs={'id': '2e88757e-fb21-4af4-9c2f-7020a436384a', 'finish_reason': 'COMPLETE', 'content': 'To answer your question, I need some additional context. Who or what are you referring to specifically?', 'token_count': {'input_tokens': 72.0, 'output_tokens': 20.0}} response_metadata={'id': '2e88757e-fb21-4af4-9c2f-7020a436384a', 'finish_reason': 'COMPLETE', 'content': 'To answer your question, I need some additional context. Who or what are you referring to specifically?', 'token_count': {'input_tokens': 72.0, 'output_tokens': 20.0}} id='run--242a7108-a792-4de9-a8dd-1e8518dbf776-0' usage_metadata={'input_tokens': 72, 'output_tokens': 20, 'total_tokens': 92}
